In [1]:
#-----------引入基本套件-----------#
import time
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
from keras import backend as K
from keras.callbacks import (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,
                             TensorBoard)
from keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import Adam
from keras.utils.data_utils import get_file
from PIL import Image   
#-----------引入模型架構-----------#
from nets.generator_training import *  #引入.fir_generator所需要使用的生成器
from nets.unet import *                         #從net/unet.py 中引入Unet模型，內有Unet3、4、5、6
from nets.FCN8 import *                       #FCN8
from nets.pspnet import pspnet           #PSPNET
#-----引入損失函數&評價函數-----#
from utils.loss import *       #調用自訂義的損失函數
from utils.metrics import *  #調用自訂義的評價函數

Using TensorFlow backend.
C:\Users\Jwander\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jwander\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jwander\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jwand

In [2]:
# 範例模型為RD+IR
#---------控制輸入圖片---------#
RA = False  #True則代表輸入該圖片
RD = True
IR = True
GI = False
#---------控制雨量區間---------#
interval = 'EWB01' #EWB01共100個標籤
num_classes = 100
#---------控制移動時間--------#
shift = 1                        #預測 t+shift 時刻
t2t = 't2t+'+str(shift)
#--------控制訓練的參數------#
choose_model = 'Unet3'   #模型名稱

In [3]:
"""#predict 【針對單一輪次輸出預測(圖像、陣列)】
#EWB01、EWB10要使用unet.detect_image_merge_label輸出二維陣列到output/predict_array資料夾中
並在計算完輸出CWB18的雨量分布圖做可視化的參考
"""
from utils.model_output import *
from PIL import Image
import glob
import os
import pandas as pd
if not os.path.isdir('./output'):  #若該資料夾不存在則生成資料夾
    os.mkdir('./output')
if not os.path.isdir('./output/predict'):
    os.mkdir('./output/predict')
if not os.path.isdir('./output/predict_array'):
    os.mkdir('./output/predict_array')  
checkpoint = glob.glob('./checkpoint/*.h5')  #glob取出checkpoint所有檔名
for last_checkpoint in checkpoint:  #走訪checkpoint
    None #走完便會將最後一個checkpoint檔名存入llast_checkpoint中
print(last_checkpoint)
#last_checkpoint = './checkpoint/ep075-loss0.624-val_loss1.536.h5' #手動調用想要觀察的模型

if __name__ == '__main__':
    K.clear_session()  #清空權重非常重要
    blend = False# blend參數用於控制是否讓識別結果和原圖混合  False不混合(輸出純白背景)

    if RD==True and IR==False and RA==False:   #RD
        model_image_size = [128,128,3]
    elif RD==False and IR==False and RA==True:   #RA
        model_image_size = [128,128,3]
    elif RD==False and IR==True and RA==False:  #IR
        model_image_size = [128,128,3]
    elif RD==True and IR==True and RA==False:  #RD+IR
        model_image_size = [128,128,4]
    elif RD==True and IR==False and RA==True:  #RD+RA
        model_image_size = [128,128,6]
    elif RD==False and IR==True and RA==True:  #IR+RA
        model_image_size = [128,128,4]
    elif RD==True and IR==True and RA==True:  #RD+IR+RA
        model_image_size = [128,128,7]
    if GI == True:
        model_image_size[2] = model_image_size[2] +2  #channel+2，GI加在最後一層
    
    model_path = last_checkpoint  #最佳權重
    
    #變數unet 和 函數Unet()僅為代稱，詳情參見model_output.py，內部整合了所有模型的輸出方式
    #將根據choose_model用於選取對應的模型參數
    unet = Unet(model_path=model_path,model_image_size=model_image_size,num_classes=num_classes,
                blend=blend,RD=RD,IR=IR,GI=GI,RA=RA,choose_model=choose_model)
    
    val = pd.read_csv('./database/train_list/'+t2t+'/val.txt',header=None,delimiter=';') #讀取val資料夾用於對應計算
    pre=val[:][0]  #要預測的檔名於第0行中
    for jpg in range(len(pre)):
        RD_image = Image.open("./database/train_data_RD/val/"+pre[jpg])#匯入雷達圖像
        IR_image = Image.open("./database/train_data_IR/val/"+pre[jpg])#匯入IR圖像
        RA_image = Image.open("./database/train_data_RA/"+interval+"/val/"+ pre[jpg]) #匯入雨量分布圖(標籤)
        
        #反正都丟進去，內部在根據RA+RD+IR+GI的組合去判斷要使用哪些資料
        pre_tmp = pre[jpg] #取出第k個標籤，加上t+shift
        pre_tmp = pre_tmp[:-4]+'_t+'+str(shift)  #將標籤的檔名後四位刪除 (刪除.png) 因為標籤為csv形式
        
        #detect_image_merge_label用於輸出預測出來的陣列
        r_label = unet.detect_image_merge_label(RD_image,IR_image,RA_image)
        np.savetxt("./output/predict_array/"+pre_tmp+'.csv', r_label, delimiter=",",fmt='%d',) #預測的值儲存成csv

./checkpoint\Unet3_val_min_RMSE.h5







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
./checkpoint\Unet3_val_min_RMSE.h5 model loaded.


In [4]:
"""
計算【單一輪次的誤差】(標籤轉相對真值與真值相減)，讀取的是predict_array內的資料(陣列)
"""
#----------將標籤映射到CWB的顏色----------#
import matplotlib
nws_precip_colors = [
    "#fdfdfd",  # (253,253,253)   #white 
    "#c1c1c1",  # (193,193,193)  #grays 
    "#99ffff",  # (153,255,255)   #blue 
    "#00ccff",  # (0,204,255)
    "#0099ff",  # (0,153,255)
    "#0166ff",  # (1,102,255)
    "#329900",  # (50,153,0)        #green
    "#33ff00",  # (51,255,0)
    "#ffff00",  # (255,255,0)       #yellow
    "#ffcc00",  # (255,204,0)
    "#fe9900",  # (254,153,0)  
    "#fe0000",  # (254,0,0)          #red
    "#cc0001",  # (204,0,1)
    "#990000",  # (153,0,0)
    "#990099",  # (153,0,153)     #purple
    "#cb00cc",  # (203,0,203)
    "#ff00fe",  # (255,0,254)
    "#feccff"   # (254,204,255)
]
precip_colormap = matplotlib.colors.ListedColormap(nws_precip_colors)
eplison = 1e-7 #將所有標籤加上一個極小值，防止標籤=1的情況判斷成2
#(因為0.1<=x<1；1<=x<2，所以標籤1會歸類成2，也就是灰色變成淺藍色了)
clevels = [0,0.1, 1+eplison, 2+eplison, 6+eplison, 10+eplison, 15+eplison, 20+eplison
           , 30+eplison, 40+eplison, 50+eplison, 70+eplison, 90+eplison,110+eplison
           ,130+eplison,150+eplison, 200+eplison,300+eplison,500+eplison]  #自定義顏色列表
norm = matplotlib.colors.BoundaryNorm(clevels, 18)
#----------計算累積雨量和RMSE----------#
val = pd.read_csv('./database/train_list/'+t2t+'/val.txt',header=None,delimiter=';') #讀取val資料夾用於對應計算
pre=val[:][0]  #在輸出預測已經將t->t+1的情形考慮進去，輸出了檔名皆對應 
for k in range(len(pre)):
    pre_tmp = pre[k] #取出第k個標籤，取出名稱並加上t+shift.png
    pre_tmp = pre_tmp[:-4]+'_t+'+str(shift)+'.csv'  #shift需同predict，參照上格or資料夾
    predict = np.genfromtxt('./output/predict_array/'+pre_tmp,delimiter=',') #匯入預測出來的圖像
    #----------將標籤值映射到相對真值----------#
    plt.figure(figsize=(1,1)) #改變圖片尺寸(英吋)   #1英吋對應dpi pixel 
    plt.imshow(predict,cmap=precip_colormap, alpha=1, norm=norm)  #alpha為透明度
    plt.axis('off')
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)  #控制子圖邊框
    plt.savefig("./output/predict/"+pre_tmp[:-4]+'.png',dpi=128,pad_inches=0.0) 
    plt.close()  #關閉圖像